In [114]:
import numpy as np
import pandas as pd
from datetime import timedelta

In [3]:
cases = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_cases.csv', index_col=0)
product = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_product.csv', index_col=0)
dim_customer = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_info.csv', index_col=0)

# Generate more users:
- Generate customers in customer_cases
- Add the new customers to dim_customer and generate age, gender, 
- Add customer to products - Add created date. + Generate new cancelation month (after created date)


In [122]:
sample_size = 100000

gen_cases = pd.DataFrame({
    'case_id' :  np.array(pd.Series(np.repeat("CC", sample_size)) + pd.Series(np.arange(330613, 330613+sample_size)).astype(str)),
    'date_time' : cases['date_time'].sample(sample_size, replace = True).to_numpy(),
    'customer_id' : np.array(pd.Series(np.repeat("C", sample_size)) + pd.Series(np.arange(511380, 511380+sample_size)).astype(str)),
    'channel' : cases['channel'].sample(sample_size, replace = True).to_numpy(),
    'reason' : cases['reason'].sample(sample_size, replace = True).to_numpy(),
    })

gen_cases


,case_id,date_time,customer_id,channel,reason
0,CC330613,2021-02-12 08:15:07,C511380,phone,signup
1,CC330614,2020-01-14 08:54:13,C511381,phone,support
2,CC330615,2020-04-02 10:22:24,C511382,phone,support
3,CC330616,2021-12-17 10:54:52,C511383,phone,support
4,CC330617,2021-12-02 15:59:46,C511384,phone,support
...,...,...,...,...,...
99995,CC430608,2020-03-15 15:11:08,C611375,email,support
99996,CC430609,2021-11-03 11:32:42,C611376,phone,signup
99997,CC430610,2019-02-27 12:35:53,C611377,phone,support
99998,CC430611,2021-03-18 15:48:09,C611378,phone,signup


In [123]:
gen_cust = pd.DataFrame({
    'customer_id' : np.array(pd.Series(np.repeat("C", sample_size)) + pd.Series(np.arange(511380, 511380+sample_size)).astype(str)),
    'age' : dim_customer['age'].sample(sample_size, replace = True).to_numpy(),
    'gender' : dim_customer['gender'].sample(sample_size, replace = True).to_numpy(),
    })

gen_cust

,customer_id,age,gender
0,C511380,58,male
1,C511381,61,male
2,C511382,58,female
3,C511383,53,female
4,C511384,46,male
...,...,...,...
99995,C611375,63,male
99996,C611376,70,male
99997,C611377,54,male
99998,C611378,55,male


In [124]:
def add_random_days(date):
    n = np.random.uniform(30,365)
    new_date = date + timedelta(days=n)

    return new_date

In [137]:
gen_prod = gen_cases[['customer_id', 'date_time']].copy()

gen_prod['product'] = product['product'].sample(sample_size, replace = True).to_numpy()

gen_prod = gen_prod[['customer_id', 'product', 'date_time']].copy()

gen_prod['date_time'] = pd.to_datetime(gen_prod['date_time']) 

gen_prod['cancel_date_time'] = gen_prod['date_time']

gen_prod['cancel_date_time'] = gen_prod['cancel_date_time'].apply(lambda x: add_random_days(x))

gen_prod = gen_prod.rename(columns={'date_time': 'signup_date_time'})

gen_prod

,customer_id,product,signup_date_time,cancel_date_time
0,C511380,prd_2,2021-02-12 08:15:07,2021-07-05 11:04:55.689007
1,C511381,prd_1,2020-01-14 08:54:13,2020-03-24 11:02:19.153757
2,C511382,prd_2,2020-04-02 10:22:24,2020-07-27 07:16:58.507454
3,C511383,prd_1,2021-12-17 10:54:52,2022-08-14 15:56:45.928192
4,C511384,prd_1,2021-12-02 15:59:46,2022-11-27 22:28:17.900204
...,...,...,...,...
99995,C611375,prd_1,2020-03-15 15:11:08,2020-07-08 06:02:56.017504
99996,C611376,prd_1,2021-11-03 11:32:42,2022-06-01 14:41:45.468641
99997,C611377,prd_1,2019-02-27 12:35:53,2019-06-26 21:58:59.136712
99998,C611378,prd_1,2021-03-18 15:48:09,2021-10-24 23:21:27.647024


# Merge original and generated data

In [127]:
new_cases = pd.concat([cases,gen_cases], ignore_index=True)
new_product = pd.concat([product,gen_prod], ignore_index=True)
new_dim_customer = pd.concat([dim_customer, gen_cust], ignore_index=True)

# Save new data

In [147]:
new_cases.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_cases.csv', index=False, encoding='utf-8', sep=',')
new_product.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_product.csv', index=False, encoding='utf-8', sep=',')
new_dim_customer.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_info.csv', index=False, encoding='utf-8', sep=',')